Installations:

`conda install dask`

`pip install dask-jobqueue>=0.7.1`

`pip install ipywidgets`

This jobqueue version is important - 0.7.0 had a bug that affected passing in scheduler_options like the dashboard port

In [7]:
from CMLDask import CMLDask
from dask.distributed import wait, as_completed

In [8]:
client = CMLDask.new_dask_client("test_dask", "1GB")

Unique port for jrudoler is 51360
{'dashboard_address': ':51360'}
To view the dashboard, run: 
`ssh -fN jrudoler@rhino2.pysch.upenn.edu -L 8000:192.168.86.142:51360` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser


In [5]:
import numpy as np
from time import sleep

def add(a, b):
    sleep(np.random.randint(0, 10))
    return a + b

def error_add(a, b):
    sleep(np.random.randint(0, 10))
    if a % 2:
        raise ValueError
    return a + b

In [6]:
futures = client.map(add, range(40), range(40))
CMLDask.progress(futures)

VBox()

In [7]:
results = client.gather(futures)
np.array(results)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66,
       68, 70, 72, 74, 76, 78])

Now with errors

In [8]:
futures = client.map(error_add, range(40), range(40))

Wait for all jobs to finish, check for errors

In [9]:
wait(futures)
errors = CMLDask.get_exceptions(futures, range(40))
errors

,param,exception,traceback_obj
index,,,
1,1,ValueError(),<traceback object at 0x2b7811222f50>
3,3,ValueError(),<traceback object at 0x2b781122fa50>
5,5,ValueError(),<traceback object at 0x2b78129f5a50>
7,7,ValueError(),<traceback object at 0x2b7812c74a50>
9,9,ValueError(),<traceback object at 0x2b7812a09140>
11,11,ValueError(),<traceback object at 0x2b7812a019b0>
13,13,ValueError(),<traceback object at 0x2b78129cfe10>
15,15,ValueError(),<traceback object at 0x2b78129ac2d0>
17,17,ValueError(),<traceback object at 0x2b78128be730>


Pick out the index where you want to view the traceback message

In [8]:
CMLDask.print_traceback(errors, 5)

  File "<ipython-input-3-a11e4f4c2959>", line 11, in error_add
    raise ValueError


Notice gathering these doesn't work because there are errors

In [9]:
client.gather(futures)

ValueError: 

Instead, let's filter for successful ones

In [10]:
good_futures = [future for future in futures if future.status=='finished']
client.gather(good_futures)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76]